In [2]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy
from kcontractions import kkcontractions
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [3]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\potsh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\potsh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\potsh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
df=pd.read_csv('movie.csv')

In [5]:
df.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [6]:
df.shape

(40000, 2)

In [7]:
df['label'].value_counts()

label
0    20019
1    19981
Name: count, dtype: int64

In [8]:
df['text']=df['text'].apply(lambda x: x.lower())

In [9]:
df.head()

,text,label
0,i grew up (b. 1965) watching and loving the th...,0
1,"when i put this movie in my dvd player, and sa...",0
2,why do people who do not know what a particula...,0
3,even though i have great interest in biblical ...,0
4,im a die hard dads army fan and nothing will e...,1


In [10]:
def remove_htmls(text):
    pattern=re.compile('<.*?>')
    return pattern.sub(r'',text)

In [11]:
df['text']=df['text'].apply(lambda x: remove_htmls(x))

In [12]:
def remove_urls(text):
    pattern=re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'',text)

In [13]:
df['text']=df['text'].apply(lambda x: remove_urls(x))

In [14]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [15]:
exclude=string.punctuation

In [16]:
def remove_punc(text):
    for char in exclude:
        text=text.replace(char,'')
    return text

In [17]:
df['text']=df['text'].apply(lambda x: remove_punc(x))

In [18]:
df.head()

,text,label
0,i grew up b 1965 watching and loving the thund...,0
1,when i put this movie in my dvd player and sat...,0
2,why do people who do not know what a particula...,0
3,even though i have great interest in biblical ...,0
4,im a die hard dads army fan and nothing will e...,1


In [19]:
from nltk.corpus import stopwords

In [20]:
stopword_l=stopwords.words('english')

In [21]:
def remove_stopw(text):
    return " ".join([word for word in str(text).split() if word not in stopword_l])


In [22]:
df['text']=df['text'].apply(lambda x: remove_stopw(x))

In [23]:
df.head()

,text,label
0,grew b 1965 watching loving thunderbirds mates...,0
1,put movie dvd player sat coke chips expectatio...,0
2,people know particular time past like feel nee...,0
3,even though great interest biblical movies bor...,0
4,im die hard dads army fan nothing ever change ...,1


In [24]:
def tokens_l(text):
    tokens=nltk.word_tokenize(text)
    tokens=[token.strip() for token in tokens]
    return tokens
    

In [25]:
df['text']=df['text'].apply(lambda x: tokens_l(x))

In [26]:
df['text']

0        [grew, b, 1965, watching, loving, thunderbirds...
1        [put, movie, dvd, player, sat, coke, chips, ex...
2        [people, know, particular, time, past, like, f...
3        [even, though, great, interest, biblical, movi...
4        [im, die, hard, dads, army, fan, nothing, ever...
                               ...                        
39995    [western, union, something, forgotten, classic...
39996    [movie, incredible, piece, work, explores, eve...
39997    [wife, watched, movie, plan, visit, sicily, st...
39998    [first, watched, flatliners, amazed, necessary...
39999    [would, film, good, gross, estimated, 95000000...
Name: text, Length: 40000, dtype: object

In [27]:
wlm=WordNetLemmatizer()

In [28]:
def lemmatize_t(text):
    tokens=[wlm.lemmatize(t) for t in text]
    return " ".join(tokens)

In [29]:
df['text']=df['text'].apply(lemmatize_t)

In [30]:
df['text']

0        grew b 1965 watching loving thunderbird mate s...
1        put movie dvd player sat coke chip expectation...
2        people know particular time past like feel nee...
3        even though great interest biblical movie bore...
4        im die hard dad army fan nothing ever change g...
                               ...                        
39995    western union something forgotten classic west...
39996    movie incredible piece work explores every noo...
39997    wife watched movie plan visit sicily stromboli...
39998    first watched flatliners amazed necessary feat...
39999    would film good gross estimated 95000000 award...
Name: text, Length: 40000, dtype: object

In [31]:
df.head()

,text,label
0,grew b 1965 watching loving thunderbird mate s...,0
1,put movie dvd player sat coke chip expectation...,0
2,people know particular time past like feel nee...,0
3,even though great interest biblical movie bore...,0
4,im die hard dad army fan nothing ever change g...,1


In [32]:
print(df['text'])

0        grew b 1965 watching loving thunderbird mate s...
1        put movie dvd player sat coke chip expectation...
2        people know particular time past like feel nee...
3        even though great interest biblical movie bore...
4        im die hard dad army fan nothing ever change g...
                               ...                        
39995    western union something forgotten classic west...
39996    movie incredible piece work explores every noo...
39997    wife watched movie plan visit sicily stromboli...
39998    first watched flatliners amazed necessary feat...
39999    would film good gross estimated 95000000 award...
Name: text, Length: 40000, dtype: object


In [33]:
from sklearn.pipeline import Pipeline

In [34]:
X=df['text']


In [35]:
Y=df['label']

In [36]:
tfidf=TfidfVectorizer()

In [37]:
X_tfidf=tfidf.fit_transform(X)

In [38]:
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf,
    Y,
    test_size=0.2,
    random_state=2002,
    stratify=df.label
)

In [39]:
X_train.shape

(32000, 181626)

In [40]:
X_test.shape

(8000, 181626)

In [41]:
y_train.shape

(32000,)

In [42]:
y_test.shape

(8000,)

In [43]:
clf=Pipeline([
    ('KNN',KNeighborsClassifier())
])
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print(classification_report(y_test, y_pred))

KeyboardInterrupt: 

In [ ]:
clf=Pipeline([
    ('Multi NB', MultinomialNB())
])
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
rmc= RandomForestClassifier()

In [ ]:
rmc.fit(X_train, y_train)

In [ ]:
y_pred=rmc.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
clf=Pipeline([
    ('Linear', SGDClassifier())
])
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
clf=Pipeline([
    ('Logistic', LogisticRegression())
])
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print(classification_report(y_test, y_pred))